In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
# Set the random seed for reproducibility
tf.random.set_seed(42)

df = pd.read_csv('data.csv', usecols=[4,5])
data = df.values
timesteps = data.shape[0]

# Define the input shape and output shape
input_shape = (None, timesteps, 2)  # (batch_size, timesteps, input_dim)
output_shape = (None, 50)  # (batch_size, output_dim)

In [3]:
# Define the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=input_shape[1:]),
    tf.keras.layers.Dense(output_shape[1], activation='linear')
])

In [4]:
# Generate the output
output_data = model.predict(np.expand_dims(data, axis=0))

1/1 [==============================] - 1s 636ms/step


In [6]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]

tfmodel = converter.convert()
with open('model.tflite', 'wb') as f:
  f.write(tfmodel)
  # f.close()